In [1]:
# ============================================================
# ENVIRONMENT SETUP
# Import required libraries and configure display settings
# This ensures consistent formatting and full column visibility
# ============================================================

import pandas as pd
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

print("Environment ready.")

Environment ready.


In [2]:
# ============================================================
# DATA INGESTION
# Load segmented customer churn datasets
# Each table represents a different business domain:
# - Demographics
# - Location
# - Services usage
# - Customer status
# - Population (aggregated by geography)
# ============================================================

base_path = "../data"

demographics = pd.read_csv(os.path.join(base_path, "Telco_customer_churn_demographics.csv"))
location = pd.read_csv(os.path.join(base_path, "Telco_customer_churn_location.csv"))
population = pd.read_csv(os.path.join(base_path, "Telco_customer_churn_population.csv"))
services = pd.read_csv(os.path.join(base_path, "Telco_customer_churn_services.csv"))
status = pd.read_csv(os.path.join(base_path, "Telco_customer_churn_status.csv"))

print("Datasets loaded successfully.")

Datasets loaded successfully.


In [3]:
# ============================================================
# DATA STRUCTURE VALIDATION
# Confirm row counts and ensure consistent customer-level granularity
# Expectation:
# - 7043 rows for customer-level tables
# - Population table aggregated by location
# ============================================================

print("Demographics:", demographics.shape)
print("Location:", location.shape)
print("Population:", population.shape)
print("Services:", services.shape)
print("Status:", status.shape)

Demographics: (7043, 8)
Location: (7043, 8)
Population: (1671, 3)
Services: (7043, 29)
Status: (7043, 10)


In [4]:
for name, df in {
    "demographics": demographics,
    "location": location,
    "services": services,
    "status": status
}.items():
    print(f"{name} columns:")
    print(df.columns.tolist())
    print("------")

demographics columns:
['Customer ID', 'Gender', 'Age', 'Under 30', 'Senior Citizen', 'Married', 'Dependents', 'Number of Dependents']
------
location columns:
['Customer ID', 'Country', 'State', 'City', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude']
------
services columns:
['Customer ID', 'Quarter', 'Referred a Friend', 'Number of Referrals', 'Tenure in Months', 'Offer', 'Phone Service', 'Avg Monthly Long Distance Charges', 'Multiple Lines', 'Internet Service', 'Internet Type', 'Avg Monthly GB Download', 'Online Security', 'Online Backup', 'Device Protection Plan', 'Premium Tech Support', 'Streaming TV', 'Streaming Movies', 'Streaming Music', 'Unlimited Data', 'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charge', 'Total Charges', 'Total Refunds', 'Total Extra Data Charges', 'Total Long Distance Charges', 'Total Revenue']
------
status columns:
['Customer ID', 'Quarter', 'Satisfaction Score', 'Customer Status', 'Churn Label', 'Churn Value', 'Churn Score', 'CLTV', 'Chur

In [5]:
# ============================================================
# DATA MERGING
# Merge customer-level tables using Customer ID as primary key
# Using LEFT JOIN to preserve full customer base
# Avoid dropping customers due to missing downstream attributes
# ============================================================

df = demographics.merge(location, on="Customer ID", how="left") \
                 .merge(services, on="Customer ID", how="left") \
                 .merge(status, on="Customer ID", how="left")

print("Final merged shape:", df.shape)

Final merged shape: (7043, 52)


In [6]:
# ============================================================
# MISSING VALUE ANALYSIS
# Identify null patterns to distinguish:
# - True data quality issues
# - Business logic-driven nulls (e.g., churn reason only for churned customers)
# ============================================================

missing = df.isnull().sum().sort_values(ascending=False)
missing[missing > 0]

Churn Reason      5174
Churn Category    5174
Offer             3877
Internet Type     1526
dtype: int64

In [7]:
# ============================================================
# CONTROLLED NULL HANDLING
# Replace business-logic nulls with explicit categorical values
# Avoid arbitrary imputation
# ============================================================

df["Churn Reason"] = df["Churn Reason"].fillna("Not Churned")
df["Churn Category"] = df["Churn Category"].fillna("Not Churned")
df["Offer"] = df["Offer"].fillna("No Offer")
df["Internet Type"] = df["Internet Type"].fillna("No Internet")

print("Missing values after cleaning:", df.isnull().sum().sum())

Missing values after cleaning: 0


In [8]:
# ============================================================
# FEATURE ENGINEERING
# Create behavioral and revenue-based features
# These improve analytical depth and model signal strength
# ============================================================

# Revenue efficiency metric
df["Revenue Per Month"] = df["Total Revenue"] / df["Tenure in Months"].replace(0, 1)

# High-value segmentation flag
df["High Value Customer"] = df["Revenue Per Month"] > df["Revenue Per Month"].median()

# Engagement metric based on service adoption
service_cols = [
    "Online Security", "Online Backup", "Device Protection Plan",
    "Premium Tech Support", "Streaming TV", "Streaming Movies",
    "Streaming Music"
]

df["Engagement Score"] = df[service_cols].apply(lambda row: (row == "Yes").sum(), axis=1)

In [9]:
# ============================================================
# FEATURE VALIDATION
# Analyze distribution and relationship to churn
# ============================================================

df.groupby("Churn Label")[["Revenue Per Month", "Engagement Score"]].mean()

df[["Revenue Per Month", "Engagement Score", "Churn Value"]].corr()


,Revenue Per Month,Engagement Score,Churn Value
Revenue Per Month,1.000000,0.602871,0.170906
Engagement Score,0.602871,1.000000,-0.064526
Churn Value,0.170906,-0.064526,1.000000


In [10]:
# ============================================================
# EXPORT CLEAN DATASET
# Save fully merged, cleaned, and engineered dataset
# Ready for modeling or BI integration
# ============================================================

output_path = "../data/customer_churn_cleaned.csv"
df.to_csv(output_path, index=False)

print("Clean dataset exported successfully.")

Clean dataset exported successfully.
